<a href="https://colab.research.google.com/github/gosaitos/GEN-AI/blob/main/Wiki_bot(using_RAG).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#!pip install langchain openai wikipedia
#!pip install langchain-community


In [28]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.utilities import WikipediaAPIWrapper

# Set OpenRouter API Key
os.environ["OPENAI_API_KEY"] = ""

# Use OpenRouter API
llm = ChatOpenAI(
    openai_api_base="https://openrouter.ai/api/v1",
    model_name="mistralai/mistral-7b-instruct",  # Choose a model from OpenRouter's available options
    temperature=0.7
)


In [29]:
wiki = WikipediaAPIWrapper()

def search_wikipedia(query):
    try:
        result = wiki.run(query)
        return result
    except Exception as e:
        return f"Error fetching Wikipedia content: {e}"


In [30]:
prompt = PromptTemplate(
    input_variables=["query", "wiki_info"],
    template="You are a Wikipedia expert. Based on the query '{query}', provide a response using this Wikipedia information:\n\n{wiki_info}"
)

chain = LLMChain(llm=llm, prompt=prompt)

def wikipedia_chatbot(query):
    wiki_info = search_wikipedia(query)
    response = chain.run(query=query, wiki_info=wiki_info)
    return response


In [68]:
query = "Anna university"
response = wikipedia_chatbot(query)
print(response)


 Anna University is a public state university located in Chennai, Tamil Nadu, India, named after C. N. Annadurai, a former Chief Minister of Tamil Nadu. Established on September 4, 1978, the main campus is in Guindy.

Anna University has numerous affiliated institutions, and for a comprehensive list, you can refer to the Wikipedia page titled "List of colleges affiliated to the Anna University."

As of July 2014, there are 552 engineering colleges in Tamil Nadu affiliated to Anna University. Out of these, 14 are Government/aided colleges, 33 are Autonomous colleges, and the remaining are self-financing colleges, with four being University departments within Anna University. For more detailed information, you may find the list of engineering colleges in Tamil Nadu on this Google Document: [Link to the Google Document](https://docs.google.com/document/d/1AeYixeUOppsnmI_zeeIM_TwMQ01_OJCxltZ5gOnq4Cc/edit?usp=sharing).


In [10]:
import gradio as gr
import os
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.document_loaders import WikipediaLoader
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from google.generativeai import configure

# ✅ Set your Google Gemini API Key
GOOGLE_API_KEY = ""
configure(api_key=GOOGLE_API_KEY)

# ✅ Function to create a Wikipedia-based retriever
def create_wikipedia_retriever(query):
    try:
        loader = WikipediaLoader(query=query, lang="en")
        docs = loader.load()

        if not docs:
            return None  # No Wikipedia data found

        # Split documents into smaller chunks
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
        split_docs = text_splitter.split_documents(docs)

        # Generate embeddings using Google AI
        embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GOOGLE_API_KEY)

        # Store embeddings in FAISS
        vectorstore = FAISS.from_documents(split_docs, embeddings)

        return vectorstore.as_retriever()
    except Exception as e:
        return None  # Handle errors

# ✅ Initialize Gemini AI LLM
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=GOOGLE_API_KEY)

# ✅ Function to get a Wikipedia-based response
def get_response(user_query):
    retriever = create_wikipedia_retriever(user_query)

    if not retriever:
        return "No relevant data found on Wikipedia. Try another query."

    qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)
    response = qa_chain.run(user_query)

    return response if response else "Could not generate a meaningful response."

# ✅ Gradio UI
def chatbot_ui(user_input):
    return get_response(user_input)

# ✅ Launch Gradio Interface
iface = gr.Interface(
    fn=chatbot_ui,
    inputs=gr.Textbox(lines=2, placeholder="Ask me anything from Wikipedia..."),
    outputs="text",
    title="Wikipedia Chatbot (Gemini AI)",
    description="A chatbot that retrieves Wikipedia information using Gemini AI and RAG.",
    theme="compact"
)

iface.launch(share=True)  # 🔗 Generates a public link


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/gradio/blocks.py:1115: UserWarning: Cannot load compact. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/compact (Request ID: Root=1-67ee1730-2db453c221ffd7ad2c41ed1b;c3f20998-75f6-4b53-bc31-6eac14c18566)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://24bfc63ef23baeb162.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
